# Compare Search Results

Compare SERPs Logged-in vs. Incognito and Desktop vs. Mobile, screenshot and save html.

## Switch this cell to `Code` and run to install dependencies
!pip install nest_asyncio playwright playwright-stealth

In [ ]:
import random
import getpass
import asyncio
import logging
import platform
import nest_asyncio
from pathlib import Path
from playwright.async_api import async_playwright
from playwright_stealth import stealth_async

# Apply nest_asyncio to run in Jupyter
nest_asyncio.apply()

# Set up logging
logging.basicConfig(level=logging.INFO)

# User data folder configuration
username = getpass.getuser()
system = platform.system()

# Common paths
windows_base_path = f"C:\\Users\\{username}\\AppData\\Local\\Google\\Chrome\\User Data"
macos_base_path = f"/Users/{username}/Library/Application Support/Google/Chrome"
linux_base_path = f"/home/{username}/.config/google-chrome"

if system == "Windows":
    chrome_path = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
    user_data_dir = f"{windows_base_path}\\Person 1"
    local_download_path = f"C:\\Users\\{username}\\Downloads"
elif system == "Darwin":  # macOS
    chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
    user_data_dir = f"{macos_base_path}/Person 1"
    local_download_path = f"/Users/{username}/Downloads"
elif system == "Linux":  # Ubuntu/Debian
    chrome_path = "/usr/bin/google-chrome"
    user_data_dir = f"{linux_base_path}/Person 1"
    local_download_path = f"/home/{username}/Downloads"
else:
    raise Exception("Unsupported operating system")

# Set up keywords to search
keywords = """
foo
resume margins
""".strip().split("\n")

# Create output directories if they don't exist
html_output = Path("html")
png_output = Path("pngs")
html_output.mkdir(parents=True, exist_ok=True)
png_output.mkdir(parents=True, exist_ok=True)

async def random_delay():
    await asyncio.sleep(random.uniform(1, 3))

async def search_and_capture(keywords, user_data_dir=None, device=None, context_name='default'):
    async with async_playwright() as plwt:
        if user_data_dir:
            # Logged-in browser
            context = await plwt.chromium.launch_persistent_context(
                user_data_dir,
                executable_path=chrome_path,
                headless=False,
                accept_downloads=True,
                downloads_path=local_download_path,
                ignore_default_args=["--enable-automation"],
                user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
            )
        else:
            # Incognito mode using the same Chrome executable
            context = await plwt.chromium.launch_persistent_context(
                None,
                executable_path=chrome_path,
                headless=False,
                accept_downloads=True,
                downloads_path=local_download_path,
                ignore_default_args=["--enable-automation"],
                user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
            )

        # Apply stealth settings and disable WebDriver flag for each context
        await context.add_init_script("Object.defineProperty(navigator, 'webdriver', { get: () => false })")

        for keyword in keywords:
            page = await context.new_page()

            # Apply stealth settings
            await stealth_async(page)

            await page.goto(f'https://www.google.com/search?num=100&q={keyword}')
            await random_delay()
            safe_keyword = keyword.replace(" ", "-")
            html_path = html_output / f'{context_name}_{safe_keyword}.html'
            screenshot_path = png_output / f'{context_name}_{safe_keyword}.png'

            # Save HTML content
            html_content = await page.content()
            with open(html_path, 'w', encoding='utf-8') as file:
                file.write(html_content)
            # Save screenshot
            await page.screenshot(path=screenshot_path, full_page=True)

            logging.info(f"Saved HTML and screenshot for '{keyword}' in {context_name} mode")

        await context.close()  # Close context instead of browser to properly handle persistent context

async def main():
    # Logged-in desktop mode
    await search_and_capture(keywords, user_data_dir=user_data_dir, context_name='loggedin-desktop')
    # Incognito desktop mode
    await search_and_capture(keywords, context_name='incognito-desktop')
    # Logged-in mobile mode
    await search_and_capture(keywords, user_data_dir=user_data_dir, device="iPhone 14 Pro", context_name='loggedin-mobile')
    # Incognito mobile mode
    await search_and_capture(keywords, device="iPhone 14 Pro", context_name='incognito-mobile')

print("Starting browser automation...")
asyncio.run(main())
print("Done!")